# Deploy ML Model
https://sagemaker.readthedocs.io/en/stable/frameworks/tensorflow/using_tf.html
https://sagemaker.readthedocs.io/en/stable/frameworks/tensorflow/using_tf.html#deploy-tensorflow-serving-models
https://aws.amazon.com/blogs/machine-learning/deploy-trained-keras-or-tensorflow-models-using-amazon-sagemaker/


After a TensorFlow estimator has been fit, it saves a TensorFlow SavedModel in the S3 location defined by output_path. You can call deploy on a TensorFlow estimator to create a SageMaker Endpoint, or you can call transformer to create a Transformer that you can use to run a batch transform job.

## AWS Setup

You will need to upload two files with no extension, "credentials" and "config".

The credentials file has your API key, config is preferred region

Put the following in each, change values to yours:
<hr>
credentials

```
[default]
aws_access_key_id = AKIAXN6R7JXYxxxxxxxx 
aws_secret_access_key = 1prI4W8J254zR5/ishEilmqskaZLOnazxxxxxxxx
```
<hr>
config

```
[default]
region = us-east-1
```

In [6]:
!apt-get update -q >apt-get_install.og
!pip3 -q install awscli
!pip3 -q install boto3
!pip3 -q install sagemaker
!pip3 -q install opendatasets
!pip freeze >requirements.txt
import boto3
from google.colab import files
import os, sys, stat
import shutil

# Set the bucket name that will be used everywhere
aws_bucket = '511004593648-msds436'

# check if AWS credentials already loaded, if not prompt for upload
aws_folder = os.path.expanduser('~/.aws')
path_to_file = f'{aws_folder}/credentials'
print("AWS credentials location:", path_to_file)

if os.path.exists(path_to_file):
  print("AWS credentials found in ~/.aws/credentials")
else:
  print('-'*80)
  print("Upload your AWS credentials and config files:")
  files_uploaded = files.upload()
  os.makedirs(aws_folder, exist_ok=True)
  shutil.copy('credentials',f'{aws_folder}/credentials')
  shutil.copy('credentials',f'{aws_folder}/config')
  os.chmod(f'{aws_folder}/credentials', stat.S_IRWXU)
  os.chmod(f'{aws_folder}/config', stat.S_IRWXU)
  print("files uploaded: ",files_uploaded.keys)

# all ok? this command should work
comprehend = boto3.client(service_name='comprehend', region_name="us-east-1")
text = "Machine learning will automate jobs that most people thought could only be done by people." #~Dave Waters
print("text to perform sentiment analysis: \n",text)
comprehend.detect_sentiment(Text=text, LanguageCode='en')

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
sagemaker 2.78.0 requires boto3>=1.20.21, but you have boto3 1.18.0 which is incompatible.
awscli 1.22.73 requires botocore==1.24.18, but you have botocore 1.21.65 which is incompatible.
AWS credentials location: /root/.aws/credentials
AWS credentials found in ~/.aws/credentials
text to perform sentiment analysis: 
 Machine learning will automate jobs that most people thought could only be done by people.


{'ResponseMetadata': {'HTTPHeaders': {'content-length': '162',
   'content-type': 'application/x-amz-json-1.1',
   'date': 'Sat, 12 Mar 2022 03:19:05 GMT',
   'x-amzn-requestid': '7b9db77b-2d59-4046-8a37-8429bc084dc4'},
  'HTTPStatusCode': 200,
  'RequestId': '7b9db77b-2d59-4046-8a37-8429bc084dc4',
  'RetryAttempts': 0},
 'Sentiment': 'NEUTRAL',
 'SentimentScore': {'Mixed': 0.009578169323503971,
  'Negative': 0.01551244780421257,
  'Neutral': 0.8685871362686157,
  'Positive': 0.10632215440273285}}

## Step 1: Setup

In [16]:

import boto3
import sagemaker

boto3_sess = boto3.Session()
role = sagemaker.get_execution_role(boto3_sess)
sess = sagemaker.Session()
bucket = sess.default_bucket()


import tensorflow as tf
print(tf.__version__)  # This notebook runs on TensorFlow 1.15.x or earlier
tf_framework_version = tf.__version__

# reference: https://github.com/keras-team/keras/issues/14265
!pip install "h5py==2.10.0"
import h5py
import numpy as np

AttributeError: ignored

## Step 4. Convert TensorFlow model to an Amazon SageMaker-readable format
We skipped steps since using a *.pb protocol buffer format

## Batch Transformation

From https://sagemaker.readthedocs.io/en/stable/frameworks/tensorflow/using_tf.html#run-a-batch-transform-job 

and https://sagemaker.readthedocs.io/en/stable/overview.html#sagemaker-batch-transform

Create transformer object and run over an in input directory

In [ ]:
bucket = myBucket # The name of the S3 bucket where the results are stored
prefix = 'batch-results' # The folder in the S3 bucket where the results are stored

batch_output = 's3://{}/{}/results'.format(bucket, prefix) # The location to store the results

tf_transformer = tf_estimator.transformer(instance_count=1, instance_type='ml.m4.xlarge', output_path=batch_output)

tf_transformer.transform(data=batch_input, data_type='S3Prefix', content_type='text/csv', split_type='Line')